In [1]:
import numpy as np
from scipy.constants import G, c, pi
from copy import copy, deepcopy
import pylab, time, os, glob
import subprocess
from tempfile import mkstemp
from shutil import move, copyfile, rmtree

In [2]:
#Define Constants:

Msun = 1.98892e30    #kg

#Conversion factors:

kg_in_Msun = 1./Msun
m_in_Msun = c**2/G/Msun
s_in_Msun = c**3/G/Msun

## Set the paths here - 

In [3]:
#Add path of pnevo directory
cur_dir = '/media/miguel/localdata/gk7785/NSBH_catalog/NSBH_BYID'
#pnevo_dir = '/home/bt22824/Maya/tools/phys-maya_tools/tools/PNevo'
pnevo_dir = '/media/miguel/localdata/gk7785/Maya_tutorial/tools/PNevo'

#Add path of directory containing rpar file
rpar_dir = cur_dir + '/rpar/'

#Add path where to create simulation directory
sim_dir = '/media/miguel/localdata/gk7785/simulations' ## Change this to your scratch directory

# Executable 
executable = "mpirun -np 4 /media/miguel/localdata/gk7785/Maya_tutorial/Cactus/exe/cactus_bhns_Maya"

In [4]:
rpar_dir

'/media/miguel/localdata/gk7785/NSBH_catalog/NSBH_BYID/rpar/'

## Set the initial parameters here

In [5]:
#Set the desired mass ratio, Mass of star (in Msun) and initial separation (in total mass M)
q = 5.
Mstar =  1.35                  # Msun
M_Msun = Mstar*(1+q)           # Msun
initsep_Mtotal = 9.0            # Total Mass

# EOS parameters
eos_type = "polytrope"        
gamma_poly = 2.
n = 1./(gamma_poly-1)

rhobar=0.05  


K_poly = 110
K_poly_M = K_poly/M_Msun**2 

print("Polytropic Constant = %g Msun = %.10g M"%(K_poly, K_poly_M))

# Spin Parameters  
ax_bh = ay_bh = az_bh = 0.
ax_ns = ay_ns = az_ns = 0.

#Refinement levels - 
nlvl_in_star = 0             #Keep this 0

nlvl_outside_src = 7         
npts_cover_star = 56 # across the radius
npts_cover_bh = 48 

#Scaling Factor - 
#change this to total mass if you want M_adm=1 else let it be 1 if you want everything in solar mass units
Mscale_in_Msun = M_Msun  #Msun


##############################################
##     Do Not change from here             ###
##############################################

# Compute masses of black hole and neutron star in terms of total mass and solar mass
m_bh = q/(1.+q)
m_ns = 1./(1+q)

m_bh_Msun = m_bh*M_Msun
m_ns_Msun = m_ns*M_Msun
print("Initial Desired Masses: BH mass = %g Msun, NS mass = %.10g Msun" %(m_bh_Msun, m_ns_Msun))
print("Initial Desired Masses (K=1): BH mass = %g Msun, NS mass = %.10g Msun" %(m_bh_Msun/K_poly**0.5, m_ns_Msun/K_poly**0.5))

#Compute separation in km
initsep_M = initsep_Mtotal*M_Msun/Mscale_in_Msun
initsep_km = initsep_M*Mscale_in_Msun/m_in_Msun/1000.

print("Initial Desired Separation = %g km = %g Msun"%(initsep_km, initsep_M*Mscale_in_Msun))



#Spin in spherical units - 

def spherical_coord(x,y,z):
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z/r)
    phi = np.arctan2(y,x)
    if r==0: theta = 0.
    if x==0: phi = 0.
    return r, theta, phi

a_bh_mag, a_bh_theta, a_bh_phi = spherical_coord(ax_bh, ay_bh, az_bh)
a_ns_mag, a_ns_theta, a_ns_phi = spherical_coord(ax_ns, ay_ns, az_ns)

Polytropic Constant = 110 Msun = 1.676573693 M
Initial Desired Masses: BH mass = 6.75 Msun, NS mass = 1.35 Msun
Initial Desired Masses (K=1): BH mass = 0.643587 Msun, NS mass = 0.1287174495 Msun
Initial Desired Separation = 107.674 km = 72.9 Msun


<ipython-input-5-b36aed3f9a23>:61: RuntimeWarning: invalid value encountered in double_scalars
  theta = np.arccos(z/r)


## TOV Solver - Isotropic Coordinates

follow the Bowen-York Initial data method in https://arxiv.org/abs/1606.04881 <br>
see https://arxiv.org/abs/2404.09924 for the newest version

In [6]:
#Polytropic Equation of State
def func_pressure(rho_c, Gamma0, K0, Gamma1=0, K1=0, rho_bndry=0, eostype="Polytrope"):
    if eostype=="polytrope":
        return K0*rho_c**Gamma0
    
    ## TODO: implement piecewise polytrope
    elif eostype=="PiecewisePolytrope":
        if rho_c < rho_bndry: return K0*rho_c**Gamma0
        else: return K1*rho_c**Gamma1
    
    elif eostype=="GammaLaw":
        return (Gamma0-1.)*rho_c
    
def func_central_density(press, Gamma0, K0, Gamma1=0, K1=0, press_bndry=0, eostype="Polytrope"):
    if eostype=="polytrope":
        if press<0:
            press = 0.0
        return (press/K0)**(1./Gamma0)
    
    ## TODO: implement piecewise polytrope
    elif eostype=="PiecewisePolytrope":
        if press < press_bndry: rhoc = (press/K0)**(1.0/Gamma0)
        else: rhoc = (press/K1)**(1.0/Gamma1)
        return rhoc
    
    elif eostype=="GammaLaw":
        return (press/K0)**(1./Gamma0)
    
def func_density(press, Gamma0, K0, Gamma1=0, K1=0, press_bndry=0, eostype="Polytrope"):
    
    if press<0:
        press=0.0
    
    if eostype=="polytrope":
        rho_c = func_central_density(press, Gamma0, K0, Gamma1, K1, press_bndry, eostype)
        return rho_c + press/(Gamma0-1.)
    ## TODO: implement piecewise polytrope
    
    elif eostype=="PiecewisePolytrope":
        rho_c = func_central_density(press, Gamma0, K0, Gamma1, K1, press_bndry, eostype)
        return rho_c
    
    elif eostype=="GammaLaw":
        return press/(Gamma0 - 1.)
            

        
# TOV Star in isotropic coordinates        
class bowentov_data(object):
    
    def __init__(self, initdata):     #eqstate, rho0, rho_atm, K0, n0, K1=0,  n1=0, rho_bndry=0):

        self.eos_type = initdata['eqstate'] 
        
        self.mass = initdata['init_mass']
        self.restmass = initdata['init_restmass']
        self.riso = initdata['r_init']
        self.theta = initdata['theta']
        self.thetader = initdata['thetader']
        self.psi = initdata['psi']
        self.psider = initdata['psider']
        self.verbose = initdata['verbose']

        if self.eos_type=="polytrope":
            self.K0 = initdata['K0']
            self.Gamma0 = 1.+(1./initdata['n0'])
            self.K1 = 0.0
            self.Gamma1 = 0.0
        
        elif self.eos_type=="PiecewisePolytrope":
        
            self.K0 = initdata['K0']
            self.Gamma0 = 1.+(1./initdata['n0'])
            self.K1 = initdata['K1']
            self.Gamma1 = 1.+(1./initdata['n1'])
            
        elif self.eos_type=="GammaLaw":
            
            self.Gamma0 = (1. + 1./initdata['n0'])
            self.K0 = initdata['K0']
            self.Gamma1 = 0.0
            self.K1 = 0.0
           
        self.press_bndry = 0.0
        rho_bndry = 0.0
        rho_atm = initdata['rho_atm']
        self.press_atm = func_pressure(rho_atm, self.Gamma0, self.K0, self.Gamma1, self.K1, rho_bndry, eostype=self.eos_type)


        if initdata['tov_type']=='tov_bowen':
            rho_c = initdata['rho0']
            self.press = func_pressure(rho_c, self.Gamma0, self.K0, self.Gamma1, self.K1, rho_bndry, eostype = self.eos_type)
        elif initdata['tov_type'] =='invtov_bowen':
            self.press = func_pressure(rho_atm, self.Gamma0, self.K0, self.Gamma1, self.K1, rho_bndry, eostype=self.eos_type)
           
          
        rho = func_density(self.press, self.Gamma0, self.K0, self.Gamma1, self.K1, self.press_bndry, eostype=self.eos_type)
        
        if self.verbose>0:
            print("Initial Data : \n Pressure = %g \n Density = %g \n Central Density = %g \n Psi = %g \n Theta = %g \n Isotropic Radius = %g"
                  %( self.press, rho, rho_c, self.psi, self.theta, self.riso))
        
        
    #Before calling derivatives update the pressure, density, r and m
    ## Eq.(23) in https://arxiv.org/abs/2404.09924
    def func_restmassder(self, r):

        rhoc = func_central_density(self.press, self.Gamma0, self.K0, self.Gamma1, self.K1, self.press_bndry, eostype=self.eos_type)
        restmass_der = 4.*np.pi*rhoc*(r**2.)*(self.psi**6.) 
        return restmass_der
    
    ## Eq.(22) in https://arxiv.org/abs/2404.09924
    def func_admmassder(self, r):

        rho = func_density(self.press, self.Gamma0, self.K0, self.Gamma1, self.K1, self.press_bndry, eostype=self.eos_type)
        admmass_der = 4.*np.pi*rho*(r**2.)*(self.psi**5.)
        return admmass_der
    
    def func_pder(self, r):
        rho = func_density(self.press, self.Gamma0, self.K0, self.Gamma1, self.K1, self.press_bndry, eostype=self.eos_type)
         
        if r==0.:
            pder = 0.
        else:
            pder = -1.*(rho + self.press)*((self.thetader/self.theta) - (self.psider/self.psi))
        
        return pder
    
    def func_psider(self, r):
        
        return self.psider                                        
                                           
    def func_psiderder(self, r):
        
        rho = func_density(self.press, self.Gamma0, self.K0, self.Gamma1, self.K1, self.press_bndry, eostype=self.eos_type)
        psi_dd = -2.*pi*rho*self.psi**5
        if r==0:
            psi_dd = psi_dd + 4.*pi*rho/3.
        else:
            psi_dd = psi_dd + (-1.)*2.*self.psider/r
                                           
        return psi_dd 
    
    def func_thetader(self, r):
                                           
        return self.thetader
                             
    def func_thetaderder(self, r):
        
        rho = func_density(self.press, self.Gamma0, self.K0, self.Gamma1, self.K1, self.press_bndry, eostype=self.eos_type)
        theta_dd = 2.*pi*(rho + 6.*self.press)*self.theta*(self.psi**4.)
        if r==0:
            theta_dd = theta_dd - 4.*pi*(rho + 6.*self.press)/3.
        else:
            theta_dd = theta_dd -2.*self.thetader/r
         
        return theta_dd                                  
                                           
    def __add__(self, star2):
        
        y = copy(self)
        y.press = self.press + star2.press
        y.mass = self.mass + star2.mass
        y.restmass = self.restmass + star2.restmass
        y.theta = self.theta + star2.theta                                  
        y.psi = self.psi + star2.psi
        y.thetader = self.thetader + star2.thetader                                  
        y.psider = self.psider + star2.psider                                   
        y.riso = self.riso + star2.riso
                                           
        return y
        
    def __mul__(self, k):
        
        y = copy(self)
        y.press = k*self.press
        y.mass = k*self.mass
        y.restmass = k*self.restmass
        y.psi = k*self.psi
        y.theta = k*self.theta
        y.psider = k*self.psider
        y.thetader = k*self.thetader                                   
        y.riso = k*self.riso

        return y
 
    def __rmul__(self, k):
        y = copy(self)
        y.press = k*self.press
        y.mass = k*self.mass
        y.restmass = k*self.restmass
        y.psi = k*self.psi
        y.theta = k*self.theta
        y.psider = k*self.psider
        y.thetader = k*self.thetader                                    
        y.riso = k*self.riso
      
        return y
    
    
    
class bowenstar:
    
    def __init__(self, initdata ):
        self.star = bowentov_data( initdata)
        self.delta_r = initdata['delta_r']
        self.tov_type = initdata['tov_type']
        self.verbose = initdata['verbose']
        
    def func_rk4_rhs(self, r, tov_star):
    
        y = deepcopy(tov_star)
        
        y.press = tov_star.func_pder(r)
        y.mass = tov_star.func_admmassder(r)
        y.restmass = tov_star.func_restmassder(r)
        y.psi = tov_star.func_psider(r)
        y.psider = tov_star.func_psiderder(r)    
        y.theta = tov_star.func_thetader(r)
        y.thetader = tov_star.func_thetaderder(r)                                       
        y.riso = 1.
        y.K = 0.
        y.Gamma = 0.
        
        return y
    
    def rungekuttamethod(self, tov_star, r):
        
        h = copy(self.delta_r)
        st = time.time()
        
        y = deepcopy(tov_star)
        if self.verbose==2:
            print('Copy the class object in %g'%(time.time() - st))
            st = time.time()
        
        k1 = self.func_rk4_rhs(r, y)
        if self.verbose==2:
            print('k1 done in %g'%(time.time() - st))
            st = time.time()
        
        k2 = self.func_rk4_rhs(r+h/2., y + (h/2.)*k1)
        if self.verbose==2:
            print('k2 done in %g'%(time.time() - st))
            st = time.time()
        
        k3 = self.func_rk4_rhs(r+h/2., y + (h/2.)*k2)
        if self.verbose==2:
            print('k3 done in %g'%(time.time() - st))
            st = time.time()
        
        k4 = self.func_rk4_rhs(r+h, y+h*k3)
        if self.verbose==2:
            print('k1 done in %g'%(time.time() - st))
            st = time.time()
        
        tov_star = tov_star + (h/6.)*(k1 + 2.*k2 + 2.*k3 + k4)
        rn = r + h
        if self.verbose==2:
            print('tovstar done in %g'%(time.time() - st))
            st = time.time()
        
        return [rn,tov_star]

    
    
    def tovsolver(self):
        
        i= 1
        press, admmass, restmass, psi, psider, theta, thetader,riso = [], [], [], [], [], [], [], []
        #rho_central = func_central_density(self.star.press, self.star.Gamma0, self.star.K0, self.star.Gamma1, self.star.K1, self.star.press_bndry, eostype = self.star.eos_type)
        #self.star.m = 4.0*np.pi*r**2*rho_central
        r = self.star.riso

        press.append( self.star.press)
        admmass.append(self.star.mass)
        restmass.append(self.star.restmass)
        psi.append(self.star.psi)
        psider.append(self.star.psider)
        theta.append(self.star.theta)
        thetader.append(self.star.thetader)                            
        riso.append(r)
            
        if self.verbose==1:
            print("Initial Data: Radius = %g, delta_r = %g, Pressure = %g, rho_c = %g \n"%(self.star.riso, self.delta_r, self.star.press, rho_central) )
        
        while (self.star.riso>=0):#np.abs(self.delta_r)):
      
            r, self.star = self.rungekuttamethod(self.star,r)
            press.append(self.star.press)
            admmass.append(self.star.mass)
            restmass.append(self.star.restmass)
            psi.append(self.star.psi)
            psider.append(self.star.psider)                            
            theta.append(self.star.theta)
            thetader.append(self.star.thetader)                            
            riso.append(self.star.riso)
            i=i+1
            
            if  i>500000:
                if self.verbose>0:
                    print("TOV solver not working correctly. Broken by force. ")
                break
                
            
            if (self.star.press<0):
                if self.verbose>0:
                    print("Pressure has become negative at i=%d."%i)
                break
            
        neg_press_idx = np.where(np.asarray(press)<0)
        
        if (np.size(neg_press_idx)>0):          
            
            if self.verbose>0:
                print ("Pressure goes negative, Pressure = %g at r=%g"%(press[press<0], riso[press<0]))
            last_idx=np.amin(neg_press_idx)
                
            press = np.asarray(press[:last_idx])
            admmass = np.asarray(admmass[:last_idx])
            restmass = np.asarray(restmass[:last_idx])
            psi = np.asarray(psi[:last_idx])
            psider = np.asarray(psider[:last_idx])
            theta = np.asarray(theta[:last_idx])
            thetader = np.asarray(thetader[:last_idx])                                   
            riso = np.asarray(riso[:last_idx])
            
        else:
            press = np.asarray(press)
            admmass = np.asarray(admmass)
            restmass = np.asarray(restmass)
            psi = np.asarray(psi)
            psider = np.asarray(psider)
            theta = np.asarray(theta)
            thetader = np.asarray(thetader)                                   
            riso = np.asarray(riso)

            
        
        psi_rescale = psi[-1] + psider[-1]*riso[-1]
        theta_rescale = theta[-1] + thetader[-1]*riso[-1]
        
        theta = theta/theta_rescale
        psi = psi/psi_rescale
        riso = riso*psi_rescale**2.
        lapse = theta/psi
        admmass = admmass*psi_rescale
        if self.verbose>0:
            print("\n \n Final Parameters")
            print("Radius of Star = %g"%riso[-1])
            print("Stellar  ADM mass = %g"%admmass[-1])
            print("Stellar  Rest Mass = %g"%restmass[-1])
            print("Conformal Factor at r=0 = %g and at Rstar = %g"%(psi[0],psi[-1]))
            print("Theta at r=0 = %g and at Rstar = %g"%(theta[0],theta[-1]))
            print("Pressure at r=0 = %g and at Rstar = %g"%(press[0], press[-1]))
            
        
        
        if (len(press)>1):
            return [press, admmass, restmass, psi, psider, theta, thetader, riso]
        
        else:
            print("TOV integrator failed")
            return [press, admmass, restmass, psi, psider, theta, thetader, riso]
        
       

# Obtain Initial guesses for density and radius for above massive stars

In [7]:
def construct_tov(rho_init):
    """Set up a Isotropic TOV star for a given central density
    """
    
    initdata = {}
    initdata['eqstate'] = eos_type

    gamma = gamma_poly
    K =  K_poly                                        # Msun**(2.)  

    rhoc = rho_init                                      # kg/m**3
    rhoc = rhoc/((m_in_Msun**3)*Msun)                    # Msun
   
    rho_atm = rhoc*1e-10
    press_atm = K*rho_atm**gamma

    n = 1./(gamma-1)
    delta_r = 1e-3                                       
    verbose=0

    initdata['init_mass'] = 0.
    initdata['init_restmass'] = 0.
    initdata['r_init'] = 0.
    initdata['theta'] = 1.0
    initdata['psi'] = 1.
    initdata['psider'] = 0.
    initdata['thetader'] = 0.

    initdata['K0'] = K
    initdata['n0'] = n
    initdata['K1'] = 0.
    initdata['n1'] = 0.

    initdata['rho_atm'] = rho_atm
    initdata['rho0'] = rhoc
    initdata['tov_type']='tov_bowen'
    initdata['delta_r'] = delta_r
    initdata['verbose'] = verbose

    if verbose>0:
        print("Polytropic Constants: K = %.3f (Msun^2), Gamma = %.3f"%(K, gamma))
        print("Stellar Density: rhoc = %g(/Msun^2), Pressure = %g(/Msun^2) \n"%(rhoc, K*rhoc**gamma))

    tov_bowen = bowenstar(initdata)

    press_iso, admmass_iso, restmass_iso, psi_iso, psider_iso, theta_iso, thetader_iso, r_iso = tov_bowen.tovsolver()

    if press_iso[-1]<0:
        #print('Pressure at last index is negative \n')
    
        press_iso = press_iso[:-1]
        admmass_iso = admmass_iso[:-1]
        restmass_iso = restmass_iso[:-1]
        psi_iso = psi_iso[:-1]
        psider_iso = psider_iso[:-1]
        theta_iso = theta_iso[:-1]
        thetader_iso = thetader_iso[:-1]
        r_iso = r_iso[:-1]

    radius_iso = r_iso[-1]
    mass_iso = 2*radius_iso*(psi_iso-1)
    print("Rest Mass = %g, ADM mass = %g"%(restmass_iso[-1], mass_iso[-1]))
    return radius_iso, mass_iso[-1], restmass_iso[-1]




# Find the approximate initial density and radius guess for Broyden 

In [8]:
# Initial Guess Density - This set up based on the relations between rhobar, Rbar and Mbar.

# Choosing rhobar - The basic idea is to find the stellar configuration with small radius 
# so that high resolution of stellar grid can be achieved with small number of points i.e. 
# npts_star = radius/dx_star, Since dx_star is generally fixed ~250 metres, number of 
# points covering star would depend on size of the star which we wish to keep minimum 
# to decrease the memory usage. 


rho_Msun = rhobar/(K_poly**n)
rhoc = rho_Msun*((m_in_Msun**3)*Msun)
print ("Initial density guess %g kg/m^3" % rhoc)

def get_densityguess(rho_init_old, dmbyr_old, dm_prev, rad_old, i=0):
    
    #Find radius and mass 
    var_rad_tov, var_admmass_tov, var_restmass_tov = construct_tov(rho_init_old)
    
    #Readjust the density - need to modify to bisection method
    dm_target = m_ns_Msun - var_admmass_tov
    dm_sq = (dm_target/var_rad_tov - dmbyr_old)
    
    fac=0.5
    
    rho_init_new = rho_init_old + (dm_target/fac)*rho_init_old
    if rho_init_new <0:
        raise ValueError("Density cannot be negative")
    i+=1
    
    if np.abs(dm_target)< 1e-4 :
        print("\n Density Obtained = %g and Radius = %g Msun with mass difference = %g"%(rho_init_new, var_rad_tov, dm_target))  
        return [rho_init_new,  var_rad_tov, var_restmass_tov, var_admmass_tov]
    elif np.abs(dm_target)>np.abs(dm_prev) and i>2:
        print("\n Delta M value started increasing, so stopping the iterations. rhoc = %g, delta M = %g"%(rho_init_new, dm_target))
    else:
        print("Density  = %g kg/m^3, Radius = %g Msun, delta_M = %g, delta M/R = %g, delta(dM/R) = %g"%(rho_init_new, var_rad_tov, dm_target, dm_target/var_rad_tov, dm_sq))
        return get_densityguess(rho_init_new, dm_target/var_rad_tov, dm_target, var_rad_tov, i)
        
        
        
rho_ns_init, radius_ns_iso,  restmass_ns,admmass_ns  = get_densityguess(rhoc, 0, 100., 0, i=0)   

rhof_ns_Msun =  rho_ns_init/((m_in_Msun**3)*Msun)
rhof_ns_bar = rhof_ns_Msun*K_poly**n
print("Finalized Initial Density = %g (kg/m^3) = %g (1/Msun^2), rhobar = %g"%(rho_ns_init,rhof_ns_Msun, rhof_ns_bar))


Initial density guess 2.80575e+17 kg/m^3
Rest Mass = 0.940002, ADM mass = 0.903648
Density  = 5.31047e+17 kg/m^3, Radius = 10.7008 Msun, delta_M = 0.446352, delta M/R = 0.0417122, delta(dM/R) = 0.0417122
Rest Mass = 1.3813, ADM mass = 1.29871
Density  = 5.85523e+17 kg/m^3, Radius = 9.29514 Msun, delta_M = 0.0512916, delta M/R = 0.00551811, delta(dM/R) = -0.036194
Rest Mass = 1.44798, ADM mass = 1.35639
Density  = 5.78038e+17 kg/m^3, Radius = 9.05025 Msun, delta_M = -0.00639199, delta M/R = -0.000706278, delta(dM/R) = -0.00622439
Rest Mass = 1.4393, ADM mass = 1.34892
Density  = 5.79289e+17 kg/m^3, Radius = 9.08238 Msun, delta_M = 0.00108254, delta M/R = 0.000119192, delta(dM/R) = 0.000825469
Rest Mass = 1.44077, ADM mass = 1.35018
Density  = 5.79084e+17 kg/m^3, Radius = 9.07724 Msun, delta_M = -0.00017677, delta M/R = -1.9474e-05, delta(dM/R) = -0.000138666
Rest Mass = 1.44053, ADM mass = 1.34997

 Density Obtained = 5.79118e+17 and Radius = 9.07796 Msun with mass difference = 2.90482e

# Compute the radius of the BH and NS

In [9]:
#Isotropic Radius of BH

radius_bh_iso = m_bh_Msun/2.
radius_bh_sch = m_bh_Msun*2.
print("BH radius (isotropic)     = %g Msun"%radius_bh_iso)
print("BH radius (schwarzschild) = %g Msun"%radius_bh_sch)

radius_ns_sch = radius_ns_iso*(1. + m_ns_Msun/2./radius_ns_iso)**2
print("NS radius (isotropic)     = %g Msun"%radius_ns_iso)
print("NS radius (schwarzschild) = %g Msun"%radius_ns_sch)


BH radius (isotropic)     = 3.375 Msun
BH radius (schwarzschild) = 13.5 Msun
NS radius (isotropic)     = 9.07796 Msun
NS radius (schwarzschild) = 10.4781 Msun


# Rescale the radius in units of total mass and in metres and print the source information

In [10]:
# As we use isotropic coordinates, it is better to only look at all quantities in these 
# coordinates. Also, conversion of tidal radius from isotropic to Schwarzschild is not exactly clear

radius_bh_iso_M = radius_bh_iso/Mscale_in_Msun
radius_ns_iso_M = radius_ns_iso/Mscale_in_Msun

radius_bh_iso_m = radius_bh_iso/m_in_Msun
radius_ns_iso_m = radius_ns_iso/m_in_Msun


rtidal_iso_Msun = 2.4*(q**(-2./3))*(radius_ns_iso/m_ns_Msun)*m_bh_Msun
rtidal_iso_m = rtidal_iso_Msun/m_in_Msun

r_isco_sch = 6.*m_bh_Msun
r_isco_sch_m = r_isco_sch/m_in_Msun
r_isco_iso_m = 0.5*(r_isco_sch - m_bh_Msun + r_isco_sch*(1. - 2.*m_bh_Msun/r_isco_sch)**0.5)/m_in_Msun

print("Tidal Radius (Iso Coord) = %g km, ISCO = %g km"%(rtidal_iso_m/1000., r_isco_iso_m/1000.))

print("BH:")
print("\t Radius (Isotropic) = %.10g km \n \t Mass = %g Msun\n"\
      %(radius_bh_iso_m/1000,  m_bh_Msun))

print("Star:")
print("\t rhoc = %.15g kg/m^3 \n \t Radius (Isotropic) = %.10g km \n \t Mass = %g Msun"\
      %(rho_ns_init, radius_ns_iso_m/1000, m_ns_Msun))
print("\t Compactness (iso) = %g \n"%((radius_ns_iso/m_ns_Msun)**-1))
print("\t Compactness (sch) = %g \n"%((radius_ns_sch/m_ns_Msun)**-1))

print("Separation = %g M (%g km)"%(initsep_M, initsep_km))
if initsep_M < (radius_bh_iso_M + radius_ns_iso_M)*1.2:
    raise ValueError("Initial separation too small compared with sizes of star. Please increase the separation!")


Tidal Radius (Iso Coord) = 55.0264 km, ISCO = 49.3453 km
BH:
	 Radius (Isotropic) = 4.984888055 km 
 	 Mass = 6.75 Msun

Star:
	 rhoc = 5.79118010565099e+17 kg/m^3 
 	 Radius (Isotropic) = 13.40817933 km 
 	 Mass = 1.35 Msun
	 Compactness (iso) = 0.148712 

	 Compactness (sch) = 0.12884 

Separation = 9 M (107.674 km)


## Compute Momenta from PNEvo
PNEvo: Post Newtonian Evolution to get initial orbital momenta of NS and BH 

In [11]:
##### Common Functions for PNEvo data #####
def read_data(mparfile, data):

    datafile = open(mparfile,'r')
    datafile.seek(0)
    for line in datafile.readlines():
        if data in line:
            break
    line = line.split()
    data_value = (line[-1]).split(';')[0]
    datafile.close()
    return data_value

def replace_data(file_path, pattern, subst, verbose=False):
    
    fh, abs_path = mkstemp()
    with os.fdopen(fh,'w') as new_file:
        with open(file_path) as old_file:
            pattern_found=0
            for line in old_file:
                if pattern_found==1:
                    new_file.write(line)#.replace(pattern, subst))
                elif pattern in line:
                    if verbose: print (pattern_found)
                    pattern_found=1
                    if verbose: print (pattern_found) , line
                    line_split = line.split('#')[0]
                    line_parts = (line_split.split('=')[-1]).split()
                    value = line_parts[0]
                    new_file.write(line.replace(value, subst))
                else:
                    new_file.write(line)#.replace(pattern, subst))
                
    #Remove original file
    os.remove(file_path)
    #Move new file
    move(abs_path, file_path)


def replace_data_mparfile(file_path, pattern, subst, verbose=False):
    
    fh, abs_path = mkstemp()
    with os.fdopen(fh,'w') as new_file:
        with open(file_path) as old_file:
            pattern_found=0
            for line in old_file:
                if pattern_found==1:
                    new_file.write(line)#.replace(pattern, subst))
                elif pattern in line:
                    if verbose: print (pattern_found)
                    pattern_found=1
                    if verbose: print (pattern_found) , line
                    line_split = line.split('#')[0]
                    line_parts = (line_split.split('=')[-1]).split()
                    value = line_parts[0]
                    
                    line_split = line.split()
                    idx = line_split.index(value)
                    line_split[idx] = subst
                    line_split.append('\n')
                    new_file.write(' '.join(line_split))
                else:
                    new_file.write(line)#.replace(pattern, subst))
                
    #Remove original file
    os.remove(file_path)
    #Move new file
    move(abs_path, file_path)
                

                
    

In [18]:
##### Create mpar file for PNevo #####

#Enter the directory of PNEvo - Change this path
os.chdir(pnevo_dir)

params = ['par.initial_sep', 'par.final_sep', 'par.MM', 'par.mass_ratio', 'par.a1x', 'par.a1y', 'par.a1z', \
         'par.a2x', 'par.a2y', 'par.a2z', 'par.manual_p0', 'par.px0', 'par.py0', 'par.pz0', 'par.fixed_dt', \
         'par.dtfac', 'par.spinspin', 'par.radrxn', 'par.out_time', 'par.out_baumfile', 'par.out_rpar_head']

#Provide the initial parameters in terms of total mass =1
par_initvalues = {'par.initial_sep':100,           # Initial Separation
                  'par.final_sep': initsep_Mtotal,     # Final Separation
                  'par.MM':1,                     # Total Mass (remains 1)
                  'par.mass_ratio':q,             # Mass Ratio of binary
                  'par.a1x':ax_bh,                # Dimensionless spin of BH1
                  'par.a1y':ay_bh,
                  'par.a1z':az_bh,
                  'par.a2x':ax_ns,              # Dimensionless spin of BH2
                  'par.a2y':ay_ns,
                  'par.a2z':az_ns, 
                  'par.manual_p0':0,    # Change to 1 if manually specifying the momentum of BH1 (p_BH2 = -1*p_BH1)
                  'par.px0':0.0,          # Momentum of BH1 (If 0, will be evaluated by PNevo) 
                  'par.py0':0.0, 
                  'par.pz0':0,
                  'par.fixed_dt':0,     # If 1, timestep = fixed_dt. If 0, time step adjusted internally based on separation
                  'par.dtfac':25,       # Timestep of evolution. Set fixed_dt to 1 if you want to specify the timestep  here.
                  'par.spinspin':1,     # Account for spin-spin interactions (default is 1)
                  'par.radrxn':1,       # Account for Radiation reaction (default is 1)
                  'par.out_time':0,     # Output time
                  'par.out_baumfile':1, # Produced Maya parfile for use with Baum Solver (default =1)
                  'par.out_rpar_head':1} #Create file with all initial parameters for rpar file (default = 1)


if os.path.exists('./initdata_NSBH.mpar'):
    print("PNEvo Mpar file found")
else:
    raise ValueError("PNEvo mpar file initdata_NSBH.mpar missing. ")
    
    
for p in params:
    replace_data_mparfile('initdata_NSBH.mpar', p, str(par_initvalues[p]))
    



PNEvo Mpar file found


In [18]:
pnevo_dir
#os.getcwd()
os.chdir(pnevo_dir)
os.getcwd()

'/media/miguel/localdata/gk7785/Maya_tutorial/tools/PNevo'

In [19]:
##
## Run Post Newtonian evolution script - PNevo, and save the momenta from output


output = subprocess.check_output("./PNevo initdata_NSBH.mpar",shell=True, universal_newlines=True)
print (output)

for line in output.splitlines():
   
    if "$ppx" in line:
        px_bh = float((line.split("=")[-1]).split(';')[0])
    
    elif "$ppy" in line:
        py_bh = float((line.split("=")[-1]).split(';')[0])


#Compute momentum in Msun units


px_bh_Msun = px_bh*M_Msun
py_bh_Msun = py_bh*M_Msun
px_star_Msun = -1.*px_bh*M_Msun
py_star_Msun = -1.*py_bh*M_Msun

print("x-Momentum of object 1: {}".format(px_bh_Msun))
print("y-Momentum of object 1: {}".format(py_bh_Msun))
print("x-Momentum of object 1: {}".format(px_star_Msun))
print("y-Momentum of object 1: {}".format(py_star_Msun))


-----------------------PNevo-C-v1----------------------------
Initial X  = 100.0000000000	0.0000000000	0.0000000000
Initial P  = 0.0000000000	0.0141692973	0.0000000000
Initial S1 = 0.0000000000	0.0000000000	0.0000000000
Initial S2 = 0.0000000000	0.0000000000	0.0000000000
|S1|/m1^2 = 0.0000000000	|S2|/m2^2 = 0.0000000000
m1 = m+ = 0.83
m2 = m- = 0.17
mass ratio = 5.00
-------------------------------------------------------------

Output directory: initdata_NSBH
Output files: initdata_NSBH/initdata_NSBH_baum.par initdata_NSBH/initdata_NSBH.rpar

Time to merger from 100.0M  to 9.0M is 14810202M
Number of orbits over same range: 3701.78

phi=-1.373532191234968
alpha=theta-pi/2=0.000000000000000
th_x=0.000000000000000
phi=-1.362094468020758
alpha=theta-pi/2=0.000000000000000
th_x=0.000000000000000
#For perl script:
#----------------------------------------------------
$offset = 2.999999999999999;
$par_b  = 4.500000000000000;

$ppx     = -0.000502039329536;
$ppy     = 0.058107119400063;
$pp

## Generate rpar file for Initial Data (Broyden)

In [20]:
#Path where mixed binary files are stored - 

#Change this
os.chdir(rpar_dir)

#Create a copy of rpar file for generating broyden parfile
# copyfile('NSBH_Updated.rpar', 'NSBH_ID_Python.rpar')
copyfile('NSBH_production.rpar', 'NSBH_ID_Python.rpar')
# copyfile('NSBH_compactness.rpar', 'NSBH_ID_Python.rpar')
# nlvl_outside_src = 3         
# npts_cover_star = 24 #across the radius
# npts_cover_bh = 15 

'NSBH_ID_Python.rpar'

In [21]:
#Modify the NSBH rpar file to create parfile for Broyden:
nsbh_id_rpar = 'NSBH_ID_Python.rpar'
perl = "perl "

#Check the compact object type is BH and TOV
parobject1 = read_data(nsbh_id_rpar, 'compact_object1')
parobject2 = read_data(nsbh_id_rpar, 'compact_object2')

if not(parobject1=="\"BH\"") or not(parobject2=="\"TOV\"") :
    raise ValueError("This rpar file in not for NSBH. Please check the correct rparfile")


init_params = { "my $M_in_Msun": Mscale_in_Msun,
                "my $mq": q,
                "my $mStar_Msun": m_ns_Msun,
                "my $rStar_m": radius_ns_iso_m,
                "my $rhoStar_central_kgm3": rho_ns_init,
                "my $eos_type": "\"%s\""%eos_type,
                "my $eos_k_Msun": K_poly,
                "my $eos_gamma": gamma_poly,
                "my $params_from_broyden": 0,
                "my $momentum_from_PN" : 1,
                "my $init_separation_m": initsep_km*1000.,
                "my $P1x_PN": px_bh_Msun,
                "my $P1y_PN": py_bh_Msun,
                "my $P2x_PN": px_star_Msun,
                "my $P2y_PN": py_star_Msun,
                "my $a_BH_mag" : a_bh_mag, 
                "my $a_BH_theta" : a_bh_theta, 
                "my $a_BH_phi" : a_bh_phi,
                "my $a_Star_mag" : a_ns_mag, 
                "my $a_Star_theta" : a_ns_theta, 
                "my $a_Star_phi" : a_ns_phi,
                
                # Below parameters are for evolution only, nlevels and npoints are fixed for Broyden inside rpar
                "my $nlevels_within_star" : nlvl_in_star,
                "my $nlevels_beyond_src" : nlvl_outside_src,  
#                 "my $npoints_rad_star" : npts_cover_star,
               "my $npoints_rad_star" : 48,
                "my $npoints_rad_bh" : npts_cover_bh
              }

#Replace the parameters in rpar file
for key in sorted(init_params.keys()):
    print("{}  = {}".format(key[4:], init_params[key]))

    if key=="my $rho1_central_kgm3" or key=="my $eos_type":
        replace_data(nsbh_id_rpar, key, str(init_params[key])+';', verbose=True)
    else:
        #replace_data(nsbh_id_rpar, key, str(init_params[key])+';', verbose=False)
        replace_data(nsbh_id_rpar, key, "%.15g;"%init_params[key], verbose=False)

#Generate parfile and extract name of parfile
broydenparfile_output = subprocess.check_output(["perl", nsbh_id_rpar])
broydenparfile_output = subprocess.check_output(perl+ nsbh_id_rpar,shell=True, universal_newlines=True)


for line in broydenparfile_output.splitlines():
    if "Filename" in line:
        filename = line.split("=")[-1]
        filename = filename.replace(" ","")

#Move the parfile to corresponding path - Change this
#savepath = 'parfiles/InitialData'
#move('./%s'%filename, '%s/%s'%(savepath,filename))



M_in_Msun  = 8.100000000000001
P1x_PN  = -0.004066518569241601
P1y_PN  = 0.4706676671405104
P2x_PN  = 0.004066518569241601
P2y_PN  = -0.4706676671405104
a_BH_mag  = 0.0
a_BH_phi  = 0.0
a_BH_theta  = 0.0
a_Star_mag  = 0.0
a_Star_phi  = 0.0
a_Star_theta  = 0.0
eos_gamma  = 2.0
eos_k_Msun  = 110
eos_type  = "polytrope"
0
1
init_separation_m  = 107673.58199510055
mStar_Msun  = 1.35
momentum_from_PN  = 1
mq  = 5.0
nlevels_beyond_src  = 7
nlevels_within_star  = 0
npoints_rad_bh  = 48
npoints_rad_star  = 48
params_from_broyden  = 0
rStar_m  = 13408.17933325513
rhoStar_central_kgm3  = 5.791180105650988e+17


### Run the broyden parfile to obtain the initial data 

In [22]:
import time
start_time = time.time()
#Path where simulation results will be saved - 

#Change this
os.chdir(sim_dir)

parfile_path = rpar_dir

#Obtain parfile name and parfile path
parfile = os.path.join(parfile_path, filename)

#Run the code to get the ID output from TwoPunctures
broyden_output = subprocess.check_output(executable+" "+parfile,shell=True, universal_newlines=True)
#broyden_output = subprocess.check_output("mpirun -np 4 /home/miguel/Cactus/exe/cactus_bhns /home/miguel/projects/2021_BHNSKicks/InitialData/parfiles/NSBH_D71.8_q3_rBH_3.0_rStar_15.0_M31.7_Broyden_C19.par",shell=True, universal_newlines=True)


#Remove all the checkpoints 
os.chdir(sim_dir)
check_direcs = glob.glob('./check*')
for direc in check_direcs:
    rmtree(direc)

#Save the output of simulation (stdout)
output_dir= filename.split('.par')[0]
stdout = open('./%s/stdout'%output_dir,'w')
stdout.write(broyden_output)
stdout.close()

print('Total Time taken = %g'%(time.time() - start_time))

Total Time taken = 413.187


# Read Broyden output instead of redoing Broyden

In [23]:
os.chdir(sim_dir)
output_dir= filename.split('.par')[0]
stdout = open('./%s/stdout'%output_dir,'r')
Broyden_read = stdout.read()
stdout.close()
print(Broyden_read)
broyden_output = Broyden_read

--------------------------------------------------------------------------------

       10                                  
  1   0101       ************************  
  01  1010 10      The Cactus Code V4.2.3    
 1010 1101 011      www.cactuscode.org     
  1001 100101    ************************  
    00010101                               
     100011     (c) Copyright The Authors  
      0100      GNU Licensed. No Warranty  
      0101                                 
--------------------------------------------------------------------------------

Cactus version:    4.2.3
Compile date:      Oct 04 2024 (14:27:23)
Run date:          Feb 28 2025 (18:25:36-0600)
Run host:          uxmal.ph.utexas.edu (pid=3432605)
Working directory: /media/miguel/localdata/gk7785/simulations
Executable:        /media/miguel/localdata/gk7785/Maya_tutorial/Cactus/exe/cactus_bhns_Maya
Parameter file:    /media/miguel/localdata/gk7785/NSBH_catalog/NSBH_BYID/rpar/NSBH_D107.7_q5_rBH_5.0_rStar_13.4_M54.8


## Generate the final par file for evolution

In [24]:
#Find the last broyden iteration from output

last_iter_broyden=0
last_line_number=0
for i,line in enumerate(broyden_output.splitlines()):
    if "(Broyden): iteration #" in line:
        last_iter_broyden = int((line.split('#')[1])[0])
        last_line_number=i

if (last_iter_broyden==0):
    raise RuntimeError("Broyden Failed. Please run manually to check for errors.")
        
#Extract the final parameters from output
for i,line in enumerate(broyden_output.splitlines()):
    if i>last_line_number:
        
        if "(BowenID): TOV #1" in line:
            #These still include the scaling mass factors
            #bowen_tov_rho_cgs = float((line.split('rho_central = ')[1]).split('*')[0])
            bowen_tov_radius = float((line.split('Radius = ')[1]).split('*')[0])
        elif "(Broyden): Setting bowenid::tov_rho_central" in line:
            br_tov_rho_Kunits = float(line.split(' = ')[-1]) # Density in polytropic units
            br_tov_rho_M = br_tov_rho_Kunits/K_poly_M        # Density in Geometric units
            br_tov_rho_si = (br_tov_rho_M/Mscale_in_Msun**2)*((m_in_Msun**3)*Msun)
            print("Density (polytropic units) = %g, Density (SI) = %g"%(br_tov_rho_Kunits, br_tov_rho_si))#(br_tov_rho/Mscale_in_Msun**2)*((m_in_Msun**3)*Msun)))
        elif "(Broyden): Setting bowenid::bh_bare_mass" in line:
            br_punc_mass_M =  float(line.split(' = ')[-1])
            print("Puncture mass (in M) = %.8g"%(br_punc_mass_M))
            
            

# Warning - Do not read final masses of star and BH from stdout of Broyden. As we already provide target masses to 
# the most accurate value initially, our original values do not contain any error  and hence, would not induce
# any errors in other parameters when used for rescaling in rpar. 
        #elif "(Broyden): TOV Mass" in line:
        #    br_tov_mass = float(line.split(':')[-1])
        #elif "(Broyden): Found mass of black hole" in line:
        #    br_bh_mass = float(line.split('=')[-1])
                
print (bowen_tov_radius*Mscale_in_Msun/m_in_Msun)
bowen_tov_radius = bowen_tov_radius*Mscale_in_Msun/m_in_Msun
br_puncmass_Msun = br_punc_mass_M*Mscale_in_Msun


#Switch to rpar file paths
#Change this path
os.chdir(rpar_dir)

#Define Evolution parfile paths
nsbh_evol_rpar = 'NSBH_Evolution_Python.rpar'
nsbh_id_rpar_new = 'NSBH_ID_Python.rpar'
copyfile(nsbh_id_rpar_new, nsbh_evol_rpar)

#Replace the broyden parameters in rpar file
broyden_params = {"my $rhoStar_central_kgm3_br":br_tov_rho_si,
                  "my $puncmass_Msun_br":br_puncmass_Msun,
                  "my $radius_minus_m_br": bowen_tov_radius,
                  "my $params_from_broyden": 1}
for key in broyden_params.keys():
    print("%s: %.15g"%(key, broyden_params[key]))
    replace_data(nsbh_evol_rpar, key, "%.15g;"%broyden_params[key], verbose=False)


#Generate parfile and extract name of parfile
evolparfile_output = subprocess.check_output(["perl", nsbh_evol_rpar])

exit=evolparfile_output.decode()
print(exit)

Puncture mass (in M) = 0.82519
Density (polytropic units) = 0.112493, Density (SI) = 6.31256e+17
13081.861663151409
my $rhoStar_central_kgm3_br: 6.31255580483468e+17
my $puncmass_Msun_br: 6.684039
my $radius_minus_m_br: 13081.8616631514
my $params_from_broyden: 1
 |S_BH|  ~= 0 (j_plus = 0),   S=(0,0,0)
BH-NS System:
     M = 8.1 M_sun (q=5).
     M_BH = 0.833333333333333 M (6.75 Msun), M_NS=0.166666666666667 M (1.35 Msun).
     R_BH(Iso) =  0.416666666666667 M(4984.7237421634 m), R_Star = 1.09349604409854 M (13081.8616631514 m) 
     Initial separation = 9 M
     x_BH = 1.5 M, x_NS = -7.5 M
     Tidal radius ~ 4.4876446396671 M
     ISCO radius ~ 4.12457478565265 M
     p_BH = (-0.000502039329536, 0.058107119400063) M, p_Star = (0.000502039329536, -0.058107119400063) M.
     S_BH = (0, 0, 0) M, S_Star = (0, 0, 0) M.
     Final Time: 2000 M.

Equation of State Parameters:
     rho1_Star = 0.0670903263985183 /M^2 = 0.00102256251178964/Msun^2
     Gamma_* = 2
     K_* = 1.6765736930346 /M

In [25]:
print(0.166666666666667/1.09349604409854)

0.15241634166501647
